# Demo Application - Customized Image Generation

Gradio demo application cho style transfer.


In [ ]:
!pip install -q diffusers==0.30.0 transformers accelerate safetensors gradio

In [ ]:
# Khai báo promt
style_prompts = {
    "Action_painting":      "an action painting with energetic brush strokes of the scene",
    "Analytical_Cubism":    "an analytical cubism interpretation of the scene",
    "Contemporary_Realism": "a contemporary realism painting of the scene",
    "New_Realism":          "a new realism painting of the scene",
    "Synthetic_Cubism":     "a synthetic cubism painting of the scene",
}

STYLE_CLASSES = list(style_prompts.keys())

#  Mapping: style → file LoRA tương ứng trên /content
style_lora_files = {
    "Action_painting":      "action_paiting.safetensors",
    "Analytical_Cubism":    "cubism.safetensors",
    "Contemporary_Realism": "contemporary.safetensors",
    "New_Realism":          "newrealism.safetensors",
    "Synthetic_Cubism":     "synthetic.safetensors",
}

#  Negative prompt cố định
NEG_PROMPT = (
    "blurry, low quality, distorted, bad anatomy, extra limbs, extra fingers, "
    "missing fingers, broken details, artifacts, noisy, oversaturated, deformed, "
    "low resolution"
)


In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

#Load base model
base_model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None,
).to(device)

if device == "cuda":
    pipe.enable_attention_slicing()
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception:
        pass

#  Load 5 LoRA tương ứng, mỗi cái 1 adapter_name = tên style
LORA_DIR = "/content"
for style_name, fname in style_lora_files.items():
    full_path = f"{LORA_DIR}/{fname}"
    print(f"Loading LoRA for {style_name} from {full_path}")
    pipe.load_lora_weights(
        LORA_DIR,
        weight_name=fname,
        adapter_name=style_name,
    )

print("Loaded base model + 5 LoRA adapters")


In [ ]:
# Hàm suy luận: tự chọn prompt + LoRA theo class
import numpy as np
from PIL import Image

adapter_names = list(style_lora_files.keys())

def run_style_transfer(
    input_image,
    style_class,       # "Baseline" hoặc 1 trong 5 style kia
    lora_weight,
    denoise_strength,
    guidance_scale,
    num_steps,
    seed,
):
    if input_image is None:
        return None

    if isinstance(input_image, np.ndarray):
        input_image = Image.fromarray(input_image).convert("RGB")

    input_image = input_image.resize((512, 512))

    #  Prompt tương ứng class
    prompt = style_prompts[style_class]

    #  Chỉ bật LoRA đúng style_class, các LoRA khác weight = 0
    weights = [
        float(lora_weight) if name == style_class else 0.0
        for name in adapter_names
    ]
    pipe.set_adapters(adapter_names, adapter_weights=weights)

    # seed
    generator = None
    if seed != -1:
        generator = torch.Generator(device=device).manual_seed(int(seed))

    with torch.autocast(device) if device == "cuda" else torch.no_grad():
        result = pipe(
            prompt=prompt,
            image=input_image,
            strength=denoise_strength,
            guidance_scale=guidance_scale,
            num_inference_steps=num_steps,
            negative_prompt=NEG_PROMPT,
            generator=generator,
        )
    return result.images[0]


In [ ]:
import gradio as gr

css = """
#title {text-align:center; font-size:2rem; font-weight:700;}
#desc {text-align:center; opacity:0.75; margin-bottom:20px;}
"""

def show_prompt(style_class):
    return style_prompts[style_class]

with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
    gr.Markdown("<div id='title'> Multi-LoRA Style Transfer Demo</div>")
    gr.Markdown("<div id='desc'>Upload ảnh, chọn 1 trong 5 style (mỗi style là một LoRA), rồi chuyển style.</div>")

    with gr.Row():
        with gr.Column(scale=1):
            input_image = gr.Image(label="Ảnh gốc", type="numpy", height=350)

            style_class = gr.Radio(
                label="Chọn style",
                choices=STYLE_CLASSES,
                value=STYLE_CLASSES[0],
            )

            prompt_display = gr.Textbox(
                label="Prompt được dùng (tự sinh theo class)",
                interactive=False,
                lines=2,
            )

            style_class.change(
                fn=show_prompt,
                inputs=style_class,
                outputs=prompt_display,
            )

            run_btn = gr.Button(" Chuyển Style", variant="primary")

        with gr.Column(scale=1):
            output_image = gr.Image(label="Kết quả", height=350)

            with gr.Accordion(" Tùy chỉnh nâng cao", open=False):
                lora_weight = gr.Slider(
                    0.0, 1.5, value=1.0, step=0.05,
                    label="LoRA weight (độ mạnh style)",
                )
                denoise_strength = gr.Slider(
                    0.1, 0.9, value=0.45, step=0.05,
                    label="Denoise / Style strength",
                    info="0.2–0.4: giữ nội dung; 0.6–0.8: style rất mạnh",
                )
                guidance_scale = gr.Slider(
                    3, 12, value=7.5, step=0.5,
                    label="Guidance scale (CFG)",
                )
                num_steps = gr.Slider(
                    10, 50, value=30, step=2,
                    label="Số bước suy luận",
                )
                seed = gr.Number(
                    value=-1, precision=0,
                    label="Seed (-1 = random)",
                )

    run_btn.click(
        fn=run_style_transfer,
        inputs=[
            input_image,
            style_class,
            lora_weight,
            denoise_strength,
            guidance_scale,
            num_steps,
            seed,
        ],
        outputs=output_image,
    )

demo.launch(share=True, debug=True)

